# **Data Visualization Notebook**

## Objectives

* Answer business requirement 1:
    * Average images and variability images for each class (healthy or powdery mildew).
    * The differences between average healthy and average powdery mildew cherry leaves.
    * An image montage for each class.

## Inputs

* inputs/datasets/train
* inputs/datasets/validation
* inputs/datasets/test

## Outputs

* Image shape embeddings pickle file.
* Mean and variability of images per label plot.
* Plot to distinguish the contrast between mildew infected leaves and non-infected images.
* Generate code that answers business requirement 1 and can be used to build an image montage on the Streamlit dashboard.


## Additional Comments

* No additional comments.



---

# Set Data Directory

---

## Import libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
sns.set_style("white")
from matplotlib.image import imread

## Change working directory

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

In [ ]:
current_dir = os.getcwd()
current_dir

---

## Set input directories

Set train, validation and test paths.

In [ ]:
data_dir = 'inputs/datasets'
train_path = data_dir + '/train'
val_path = data_dir + '/validation'
test_path = data_dir + '/test'

## Set output directory

In [ ]:
version = 'v1'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

### Set label names

In [ ]:
# Set the labels
labels = os.listdir(train_path)
print('Label for the images are', labels)

---

# Data visualisation of image data

---

## Image shape 

Compute average images size on train set.

In [ ]:
dim1, dim2 = [], []
for label in labels:
    for image_filename in os.listdir(train_path + '/' + label):
        img = imread(train_path + '/' + label + '/' + image_filename)
        d1, d2, colors = img.shape
        dim1.append(d1)  # image height
        dim2.append(d2)  # image width

sns.set_style("whitegrid")
fig, axes = plt.subplots()
sns.scatterplot(x=dim2, y=dim1, alpha=0.2)
axes.set_xlabel("Width (pixels)")
axes.set_ylabel("Height (pixels)")
dim1_mean = int(np.array(dim1).mean())
dim2_mean = int(np.array(dim2).mean())
axes.axvline(x=dim1_mean, color='r', linestyle='--')
axes.axhline(y=dim2_mean, color='r', linestyle='--')
plt.show()
print(f"Width average: {dim2_mean} \nHeight average: {dim1_mean}")


Images size for training the model is the average from all images in the train set.

In [ ]:
image_shape = (dim1_mean, dim2_mean, 3)
image_shape

Save the image shape embeddings

In [ ]:
joblib.dump(value=image_shape ,
            filename=f"{file_path}/image_shape.pkl")

---

# Push files to Repo

* If you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
    # create here your folder
    # os.makedirs(name='')
except Exception as e:
    print(e)
